In [10]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import psycopg2

In [8]:
dbname=''
user='postgres'
password='postgres1234'
host='database-financy-1.cxoug4ik83tt.us-east-1.rds.amazonaws.com'
port=5432

In [41]:
companies_with_anomalies = [
    ("5272", "AIG", "2008", "Financial Misrepresentaion"),
    ("310522", "FNM", "2004", "Accounting Irregularities"),
    ("18926", "Q (Quest Communications)", "2002", "Accounting Fraud."),
    ("72971", "WFC", "2016", "Consumer Fraud"),
    ("1159508", "DB", "2015", "Wire Fraud"),
    ("1114446", "UBS", "2011", "Tax Evasion"),
    ("14272", "BMY", "2002", "Financial Misrepresentation"),
    ("831001", "C", "2008", "Security Fraud"),
    ("19617", "JPM", "2012", "Trading Fraud"),
    ("1131399", "GSK", "2012", "Health Care Fraud"),
    ("312070", "BARC", "2012", "Involved in the Libor rate-manipulation scandal"),
    ("1119639", "PBR", "2014", "Petrobras"),
    ("96223", "LUK", "2005", "Leucadia National (involved in Refco scandal, now part of Jefferies Financial Group, ticker 'JEF')"),
    ("18926", "LUMN", "2002", "Lumen Technologies (formerly CenturyLink)"),
    ("47217", "HPQ", "2012", "HP Inc. (involved in the aftermath of Autonomy scandal)"),
    ("1490281", "GRPN", "2012", "Groupon (accounting irregularities post-IPO)"),
    ("1119639", "PETR4.SA", "2014", "Petrobras (Brazil)"),
    ("1767582", "LKI", "2020", "Luckin Coffee (China)"),
    ("1467858", "GM", "2014", "General Motors (U.S., ignition switch scandal)"),
    ("789019", "MSFT", "2001", "Microsoft (Accounting investigations)"),
    ("27419", "TGT", "2008", "Target Corporation (Data breach)"),
    ("833444", "JCI", "2005", "Johnson Controls (Bribery scandal)"),
    ("354950", "HD", "2010", "Home Depot (Data breach)"),
    ("1300514", "LVS", "2013", "Las Vegas Sands (Money laundering)"),
    ("320193", "AAPL", "2014", "Apple Inc. (Antitrust investigations)"),
    ("70858", "BAC", "2008", "Bank of America (Mortgage fraud)"),
    ("1318605", "TSLA", "2018", "Tesla Inc. (Elon Musk's misleading tweets)"),
    ("1018724", "AMZN", "2020", "Amazon (Price gouging investigations)"),
    ("1652044", "GOOG", "2019", "Google (Antitrust investigations)"),
    ("50863", "INTC", "2014", "Intel (Antitrust investigations)"),
    ("51143", "IBM", "2005", "IBM (Bribery scandal)"),
    ("77476", "PEP", "2016", "PepsiCo (Bribery scandal)"),
    ("829224", "SBUX", "2018", "Starbucks (Racial bias scandal)"),
    ("40545", "GE", "2009", "General Electric (Accounting fraud)"),
    ("1065280", "NFLX", "2013", "Netflix (Insider trading)"),
    ("2488", "AMD", "2009", "AMD (Antitrust lawsuit)"),
    ("858877", "CSCO", "2013", "Cisco Systems (Whistleblower case)"),
    ("1341439", "ORCL", "2011", "Oracle (Bribery scandal)"),
    ("804328", "QCOM", "2018", "Qualcomm (Antitrust lawsuit)"),
    ("1065088", "EBAY", "2014", "eBay (Data breach)"),
    ("1633917", "PYPL", "2019", "PayPal (Consumer fraud)"),
    ("1512673", "SQ", "2019", "Square (Data breach)"),
    ("1759509", "LYFT", "2019", "Lyft (Labor practices investigation)"),
    ("100493", "TSN", "2019", "Tyson Foods (Labor practices investigation)"),
    ("63754", "MKC", "2018", "McCormick & Company (Price-fixing scandal)"),
    ("91419", "SJM", "2017", "J.M. Smucker Company (Misleading labeling)"),
    ("1637459", "KHC", "2019", "Kraft Heinz (Accounting scandal)"),
    ("1103982", "MDLZ", "2017", "Mondelez International (Price-fixing)"),
    ("60086", "L", "2015", "Loews Corporation (Environmental violations)"),
    ("1041061", "YUM", "2013", "Yum! Brands (Food safety scandal)"),
    ("1286681", "DPZ", "2018", "Domino's Pizza (Wage theft)"),
    ("901491", "PZZA", "2017", "Papa John's (Racial slur controversy)"),
    ("703351", "EAT", "2015", "Brinker International (Labor practices investigation)"),
    ("887596", "CAKE", "2019", "Cheesecake Factory (Wage theft)"),
    ("1013488", "BJRI", "2020", "BJ's Restaurants (Wage theft)"),
    ("1171759", "RRGB", "2018", "Red Robin Gourmet Burgers (Wage theft)"),
    ("1525769", "PLAY", "2019", "Dave & Buster's (Wage theft)")
]


In [11]:
# Import the data
conn = psycopg2.connect(dbname=dbname,
                        user=user,
                        password=password,
                        host=host,
                        port=port)
cursor = conn.cursor()

query = """
SELECT * FROM edgar_selected_tickers_table
"""
cursor.execute(query)
results = cursor.fetchall()

cursor.close()
conn.close()

In [15]:
column_names = ['end_date', 'ticker', 'reporting_frame', 'form', 'account', 'unit', 'value']

In [16]:
df = pd.DataFrame(results, columns=column_names)
df

,end_date,ticker,reporting_frame,form,account,unit,value
0,2009-07-31,AIG,dei,10-Q/A,EntityCommonStockSharesOutstanding,shares,1.345758e+08
1,2009-10-30,AIG,dei,10-Q,EntityCommonStockSharesOutstanding,shares,1.346074e+08
2,2010-01-29,AIG,dei,10-K,EntityCommonStockSharesOutstanding,shares,1.349263e+08
3,2010-04-30,AIG,dei,10-Q,EntityCommonStockSharesOutstanding,shares,1.350706e+08
4,2010-07-30,AIG,dei,10-Q,EntityCommonStockSharesOutstanding,shares,1.351263e+08
...,...,...,...,...,...,...,...
667489,2022-12-31,AMD,us-gaap,10-K,OtherReceivables,USD,2.000000e+06
667490,2023-12-30,AMD,us-gaap,10-K,OtherReceivables,USD,9.000000e+06
667491,2024-03-30,AMD,us-gaap,10-Q,UnrecordedUnconditionalPurchaseObligationBalan...,USD,3.480000e+08
667492,2024-03-30,AMD,us-gaap,10-Q,UnrecordedUnconditionalPurchaseObligationBalan...,USD,1.820000e+08


In [22]:
pivoted_df = df.pivot_table(index=['end_date', 'ticker', 'reporting_frame', 'form', 'unit'], 
                            columns='account',
                            values='value',
                            aggfunc='sum',
                            fill_value=0)
# Fill any NaN values that might be created in the process.\We have to justify why 0 is a valid value.
pivoted_df = pivoted_df.reset_index()

In [24]:
pivoted_df.head()

account,end_date,ticker,reporting_frame,form,unit,AcceleratedShareRepurchasesSettlementPaymentOrReceipt,AccountingProfit,AccountsPayable,AccountsPayableAndAccruedLiabilitiesCurrentAndNoncurrent,AccountsPayableAndOtherAccruedLiabilities,...,WagesAndSalaries,WeightedAverageLesseesIncrementalBorrowingRateAppliedToLeaseLiabilitiesRecognisedAtDateOfInitialApplicationOfIFRS16,WeightedAverageNumberDilutedSharesOutstandingAdjustment,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfShareOutstandingBasicAndDiluted,WeightedAverageNumberOfSharesContingentlyIssuable,WeightedAverageNumberOfSharesOutstandingBasic,WeightedAverageShares,WithdrawalFromContractHoldersFunds,WorkersCompensationDiscountAmount
0,1998-12-27,AMD,us-gaap,10-K,USD,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0.0,0,0
1,2006-09-30,AAPL,us-gaap,10-K,USD,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0.0,0,0
2,2006-09-30,AAPL,us-gaap,10-K/A,USD,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0.0,0,0
3,2006-12-31,AIG,us-gaap,10-K,USD,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0.0,0,0
4,2006-12-31,AIG,us-gaap,8-K,USD,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0.0,0,0


In [28]:
X=pivoted_df.iloc[:, 5:]

In [38]:
# Scale the data
scaler = StandardScaler()
X_scaled=scaler.fit(X)

In [39]:
# Train Model
model = IsolationForest(
    n_estimators=100,
    max_samples=200,
    contamination=0.01,
    max_features=1.0,
    bootstrap=False,
    n_jobs=-1, # Using available processors from computer
    random_state=42,
)
model.fit(X)

IsolationForest(contamination=0.01, max_samples=200, n_jobs=-1, random_state=42)

In [40]:
y_pred = model.predict(X)

In [43]:
pivoted_df.iloc[:,:5]

account,end_date,ticker,reporting_frame,form,unit
0,1998-12-27,AMD,us-gaap,10-K,USD
1,2006-09-30,AAPL,us-gaap,10-K,USD
2,2006-09-30,AAPL,us-gaap,10-K/A,USD
3,2006-12-31,AIG,us-gaap,10-K,USD
4,2006-12-31,AIG,us-gaap,8-K,USD
...,...,...,...,...,...
11089,2024-04-29,BAC-PO,dei,10-Q,shares
11090,2024-04-29,BAC-PP,dei,10-Q,shares
11091,2024-04-29,BAC-PQ,dei,10-Q,shares
11092,2024-04-29,BAC-PS,dei,10-Q,shares


In [45]:
X_df = pd.DataFrame(X)
y_pred_df = pd.DataFrame(y_pred)
final_df = pd.concat([pivoted_df.iloc[:,:5],X_df , y_pred_df], axis=1)
final_df.head()

,end_date,ticker,reporting_frame,form,unit,AcceleratedShareRepurchasesSettlementPaymentOrReceipt,AccountingProfit,AccountsPayable,AccountsPayableAndAccruedLiabilitiesCurrentAndNoncurrent,AccountsPayableAndOtherAccruedLiabilities,...,WeightedAverageLesseesIncrementalBorrowingRateAppliedToLeaseLiabilitiesRecognisedAtDateOfInitialApplicationOfIFRS16,WeightedAverageNumberDilutedSharesOutstandingAdjustment,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfShareOutstandingBasicAndDiluted,WeightedAverageNumberOfSharesContingentlyIssuable,WeightedAverageNumberOfSharesOutstandingBasic,WeightedAverageShares,WithdrawalFromContractHoldersFunds,WorkersCompensationDiscountAmount,0
0,1998-12-27,AMD,us-gaap,10-K,USD,0,0,0,0,0,...,0.0,0,0,0,0,0,0.0,0,0,1
1,2006-09-30,AAPL,us-gaap,10-K,USD,0,0,0,0,0,...,0.0,0,0,0,0,0,0.0,0,0,1
2,2006-09-30,AAPL,us-gaap,10-K/A,USD,0,0,0,0,0,...,0.0,0,0,0,0,0,0.0,0,0,1
3,2006-12-31,AIG,us-gaap,10-K,USD,0,0,0,0,0,...,0.0,0,0,0,0,0,0.0,0,0,1
4,2006-12-31,AIG,us-gaap,8-K,USD,0,0,0,0,0,...,0.0,0,0,0,0,0,0.0,0,0,1


In [48]:
anomalies_detected=final_df[final_df[0] == -1]
#Creating a df with the list of anomalies
anomalies_df = pd.DataFrame(companies_with_anomalies, columns=['id', 'ticker', 'year', 'description'])

In [50]:
match_anomalies = pd.merge(anomalies_detected, anomalies_df, on='ticker', how='inner')
match_anomalies[['ticker', 'year', 'description']]

,ticker,year,description
0,BAC,2008,Bank of America (Mortgage fraud)
1,BAC,2008,Bank of America (Mortgage fraud)
2,BAC,2008,Bank of America (Mortgage fraud)
3,BAC,2008,Bank of America (Mortgage fraud)
4,BAC,2008,Bank of America (Mortgage fraud)
5,BAC,2008,Bank of America (Mortgage fraud)
6,BAC,2008,Bank of America (Mortgage fraud)
7,BAC,2008,Bank of America (Mortgage fraud)
